In [2]:
import keras
keras.__version__

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.2.4'

In [3]:
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(np.unique(chars)))
# Dictionary mapping unique characters to their index in `chars`

char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [6]:
chars.index('\n')

0

In [7]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [8]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
hist = model.fit(x,y,validation_split=0.2,epochs=4,batch_size=128)

Train on 160222 samples, validate on 40056 samples
Epoch 1/4
160222/160222 [==============================] - 216s 1ms/step - loss: 1.6326 - val_loss: 1.6580
Epoch 2/4
160222/160222 [==============================] - 216s 1ms/step - loss: 1.5585 - val_loss: 1.6233
Epoch 3/4
160222/160222 [==============================] - 216s 1ms/step - loss: 1.5135 - val_loss: 1.6080
Epoch 4/4
160222/160222 [==============================] - 216s 1ms/step - loss: 1.4848 - val_loss: 1.6123


In [13]:
import random
import sys

In [17]:
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]

for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature=0.25)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

osition of the senses of the way of the same time of the same is who has have the states of the one of the fact of the order of the fared of the emong the stand of the more present to the fact that it is all the form of the discoverance of the stand of the belief of the senses of the the stand of the present to be a destray postical best we have always would any the superal postical of the more of

In [20]:
np.random.multinomial?

In [21]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 290s 1ms/step - loss: 1.9907
--- Generating with seed: "ride, and remains inexorable. eventually--the memory yields."
------ temperature: 0.2
ride, and remains inexorable. eventually--the memory yields. the sense of a science, and is not the soul and sense of predemple of man is a present of ender and the sense of the man the speak of the such a come and in the man in the are and and a science of the specient of man in the speak of man the speak of such as a stept of the speaked the sense of the such the such a desire of stand and and in the speaked the sense of the sense of the precise of the s
------ temperature: 0.5
n the speaked the sense of the sense of the precise of the sease of a might a displeased there have demons there some nother of the the same the fast we dusing the spech as it is and it is some of in indelf cound and be the discour, it is be an order of man of man the is a consciations of self there or ou

KeyboardInterrupt: 